In [9]:
import pandas as pd
import numpy as np

# Combine all the csv's into one dataframe
def compile_csv(startYear, endYear):
    df_list = []
    for i in range(startYear, endYear + 1):
        off_filename = "Data/"+ str(i) + 'Offense.csv'
        def_filename = "Data/" + str(i) + 'Defense.csv'
        off_df = pd.read_csv(off_filename)
        off_df["Unit"] = ["Offense" if pos != "LS" else "Special" for pos in off_df["Pos"]]
        def_df = pd.read_csv(def_filename)
        def_df["Unit"] = ["Defense" if (pos != "K" and pos != "P") else "Special" for pos in def_df["Pos"]]
        df_list.append(off_df)
        df_list.append(def_df)
    data = pd.concat(df_list)
    return data

df = compile_csv(2000, 2017)
# Format player name
df["Player"] = [x.split("\\")[0] for x in df["Player"]]

# Parse out Drafted (tm/rnd/yr) column
df["Drafted (tm/rnd/yr)"] = df["Drafted (tm/rnd/yr)"].where(pd.notnull(df["Drafted (tm/rnd/yr)"]), None)
df["DraftTeam"] = [x.split(" / ")[0] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftRd"] = [x.split(" / ")[1] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftRd"] = df["DraftRd"].str.replace('[a-zA-Z]+', '')
df["DraftPick"] = [x.split(" / ")[2] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftPick"] = df["DraftPick"].str.replace('[a-zA-Z_]+', '')
df = df.drop(["Drafted (tm/rnd/yr)"], axis=1)

# Convert height to inches
def convert_height(x):
    feet = x.split("-")[0]
    inches = x.split("-")[1]
    height = (int(feet) * 12) + int(inches)
    return height
df['Height'] = df['Height'].apply(convert_height)

df.to_csv('Data/All_Data.csv', index=False)

# Add conference data and one-hot encoding
teams_conference = pd.read_csv("Data/Teams-Conference.csv")

# Merge with team conference data
merged_data = df.merge(teams_conference, left_on='School', right_on='School', how='left')

# Replace NaN with None for the 'Conf' column
merged_data['Conf'] = merged_data['Conf'].where(merged_data['Conf'].notna(), None)

# Add a "No Conference" column
merged_data['No Conference'] = merged_data['Conf'].isna().astype(int)

# One-hot encode the 'Conf' column and ensure numeric values (0 or 1)
one_hot_encoded = pd.get_dummies(merged_data['Conf'], prefix='Conf').astype(int)

# Concatenate one-hot encoded columns back to the main DataFrame
merged_data = pd.concat([merged_data, one_hot_encoded], axis=1)

# Save the final DataFrame to a new CSV
merged_data.to_csv('Data/All_Data_One_Hot_Encoded.csv', index=False)

df = pd.read_csv('Data/All_Data_One_Hot_Encoded.csv')

df

,Rk,Year,Player,Pos,AV,School,College,Height,Wt,40YD,...,Conf_Ind,Conf_MAC (East),Conf_MAC (West),Conf_MWC (Mountain),Conf_MWC (West),Conf_Pac-12 (North),Conf_Pac-12 (South),Conf_SEC (East),Conf_SEC (West),Conf_Sun Belt
0,1,2000,Bashir Yamini,WR,0.0,Iowa,College Stats,75,191,4.53,...,0,0,0,0,0,0,0,0,0,0
1,2,2000,Spergon Wynn,QB,2.0,Texas State,College Stats,75,229,4.91,...,0,0,0,0,0,0,0,0,0,1
2,3,2000,James Williams,WR,4.0,Marshall,College Stats,71,180,4.59,...,0,0,0,0,0,0,0,0,0,0
3,4,2000,Bobbie Williams,OT,64.0,Arkansas,NaN,76,332,5.26,...,0,0,0,0,0,0,0,0,1,0
4,5,2000,Michael Wiley,RB,4.0,Ohio State,College Stats,71,193,4.50,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5631,145,2017,D.J. Jones,DT,NaN,Mississippi,College Stats,73,319,5.04,...,0,0,0,0,0,0,0,0,0,0
5632,146,2017,Tanoh Kpassagnon,DE,NaN,Villanova,NaN,79,289,4.83,...,0,0,0,0,0,0,0,0,0,0
5633,147,2017,Avery Moss,DE,NaN,Youngstown State,NaN,75,264,4.79,...,0,0,0,0,0,0,0,0,0,0
5634,148,2017,Derek Rivers,DE,NaN,Youngstown State,NaN,76,248,4.61,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Strip out unneeded columns and handle undrafted players
df2 = df[['Year','Pos', 'Height', 'Wt', '40YD', 'Vertical', 'BenchReps', 'Broad Jump', '3Cone', 'Shuttle','Unit', 'DraftRd']]
df2['DraftRd'] = df2['DraftRd'].fillna(8)

# impute missing values for events which players skipped with a -1
df2 = df2.fillna('-1')
df2 = df2.apply(pd.to_numeric, errors='ignore')

# calculate quartile in each event for each player within their combine year/position 
rank_40 = []
rank_vert = []
rank_bench = []
rank_broad = []
rank_3cone = []
rank_shuttle = []
for index, row in df2.iterrows():
    year = row['Year']
    position = row['Pos']
    #Calculate 40YD quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.75)
    quartile = 4
    if row['40YD'] == -1:
        quartile = 5
    elif row['40YD'] <= quartile_1:
        quartile = 1
    elif row['40YD'] <= median:
        quartile = 2
    elif row['40YD'] <= quartile_3:
        quartile = 3
    rank_40.append(quartile)
    
    #Calculate vert quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.75)
    quartile = 4
    if row['Vertical'] == -1:
        quartile = 5
    elif row['Vertical'] >= quartile_3:
        quartile = 1
    elif row['Vertical'] >= median:
        quartile = 2
    elif row['Vertical'] >= quartile_1:
        quartile = 3
    rank_vert.append(quartile)
    
    #Calculate BenchReps quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.75)
    quartile = 4
    if row['BenchReps'] == -1:
        quartile = 5
    elif row['BenchReps'] >= quartile_3:
        quartile = 1
    elif row['BenchReps'] >= median:
        quartile = 2
    elif row['BenchReps'] >= quartile_1:
        quartile = 3
    rank_bench.append(quartile)
    
    #Calculate Broad Jump quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.75)
    quartile = 4
    if row['Broad Jump'] == -1:
        quartile = 5
    elif row['Broad Jump'] >= quartile_3:
        quartile = 1
    elif row['Broad Jump'] >= median:
        quartile = 2
    elif row['Broad Jump'] >= quartile_1:
        quartile = 3
    rank_broad.append(quartile)
    
    #Calculate 3Cone quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.75)
    quartile = 4
    if row['3Cone'] == -1:
        quartile = 5
    elif row['3Cone'] <= quartile_1:
        quartile = 1
    elif row['3Cone'] <= median:
        quartile = 2
    elif row['3Cone'] <= quartile_3:
        quartile = 3
    rank_3cone.append(quartile)
    
    #Calculate Shuttle quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.75)
    quartile = 4
    if row['Shuttle'] == -1:
        quartile = 5
    elif row['Shuttle'] <= quartile_1:
        quartile = 1
    elif row['Shuttle'] <= median:
        quartile = 2
    elif row['Shuttle'] <= quartile_3:
        quartile = 3
    rank_shuttle.append(quartile)
df2['40_quartile_yr_pos'] = rank_40 
df2['vert_quartile_yr_pos'] = rank_vert
df2['bench_quartile_yr_pos'] = rank_bench
df2['broad_quartile_yr_pos'] = rank_broad
df2['3cone_quartile_yr_pos'] = rank_3cone
df2['shuttle_quartile_yr_pos'] = rank_shuttle
df2

/var/folders/yf/m10_kkm12wg1sd18gc0mp32c0000gn/T/ipykernel_2576/2889184105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DraftRd'] = df2['DraftRd'].fillna(8)
/var/folders/yf/m10_kkm12wg1sd18gc0mp32c0000gn/T/ipykernel_2576/2889184105.py:7: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2 = df2.apply(pd.to_numeric, errors='ignore')


,Year,Pos,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Unit,DraftRd,40_quartile_yr_pos,vert_quartile_yr_pos,bench_quartile_yr_pos,broad_quartile_yr_pos,3cone_quartile_yr_pos,shuttle_quartile_yr_pos
0,2000,WR,75,191,4.53,33.0,-1.0,130.0,7.09,4.18,Offense,8,2,3,5,1,3,3
1,2000,QB,75,229,4.91,34.0,-1.0,108.0,7.71,4.59,Offense,6th,2,1,5,2,4,4
2,2000,WR,71,180,4.59,36.0,-1.0,123.0,7.22,4.16,Offense,6th,3,2,5,1,4,2
3,2000,OT,76,332,5.26,29.0,28.0,97.0,7.78,4.72,Offense,2nd,2,1,1,3,2,1
4,2000,RB,71,193,4.50,-1.0,-1.0,-1.0,-1.00,-1.00,Offense,5th,1,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2017,DT,73,319,5.04,28.5,25.0,108.0,7.73,4.65,Defense,8,2,1,2,2,4,3
145,2017,DE,79,289,4.83,30.0,23.0,128.0,7.46,4.62,Defense,8,3,4,3,1,4,4
146,2017,DE,75,264,4.79,32.5,14.0,114.0,7.25,4.43,Defense,8,2,2,4,3,3,3
147,2017,DE,76,248,4.61,35.0,30.0,123.0,6.94,4.40,Defense,8,1,1,1,2,1,2


### Splitting the positions

When evaluating players combine statistics it's difficult to compare across positions. Each position group has a different set of valuable skills, and are valued differently by teams in the draft. To create more accurate models we've decided to the daya into sub groups based on their positions and weightclasses.

In [ ]:
skill = {'offense_lightweight': ['WR', 'RB'], 'defense_leightweight':['CB', 'SS', 'FS'],'offense_midweights': ['FB', 'TE'], 'defense_midweight':['OLB', 'ILB'], 'offense_heavy': ['OT', 'OG'], 'defense_heavy':['DT','DE', 'LS'], 'quarterbacks':['QB'], 'punter':['P']}

# Create a mapping of position to skill group
position_to_group = {pos: group for group, positions in skill.items() for pos in positions}

# Add a new column to df2 with the position group
df2['SkillGroup'] = df2['Pos'].map(position_to_group)

# Sort the DataFrame by SkillGroup
df2_sorted = df2.sort_values(by='SkillGroup')

# Display the first few rows of the sorted DataFrame
(df2_sorted.head())




,Year,Pos,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Unit,DraftRd,40_quartile_yr_pos,vert_quartile_yr_pos,bench_quartile_yr_pos,broad_quartile_yr_pos,3cone_quartile_yr_pos,shuttle_quartile_yr_pos,SkillGroup
91,2015,DE,77,235,4.64,36.5,24.0,125.0,-1.00,-1.00,Defense,NaN,1,1,2,1,5,5,defense_heavy
64,2002,DT,78,320,5.09,-1.0,-1.0,-1.0,-1.00,-1.00,Defense,NaN,3,5,5,5,5,5,defense_heavy
82,2006,DT,75,301,5.27,-1.0,25.0,-1.0,-1.00,-1.00,Defense,NaN,4,5,3,5,5,5,defense_heavy
81,2006,DE,78,284,4.82,35.5,28.0,113.0,7.67,4.49,Defense,NaN,3,2,2,3,4,3,defense_heavy
78,2006,DE,74,270,4.75,-1.0,30.0,-1.0,-1.00,-1.00,Defense,8.0,2,5,1,5,5,5,defense_heavy


### Model: (insert number)

A decision tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming df2 is already loaded and contains 'Pos' and 'DraftRound' columns

def clean_draft_round(df):
    """
    Clean the DraftRound column by stripping non-numeric characters, converting to numeric, 
    and setting NaN values to 100.
    """
    # Ensure the 'DraftRd' column is treated as string
    df['DraftRd'] = df['DraftRd'].astype(str)
    
    # Extract numeric characters
    df['DraftRd'] = df['DraftRd'].str.extract('(\d+)')
    
    # Convert to numeric and set errors to NaN, then fill NaN values with 100
    df['DraftRd'] = pd.to_numeric(df['DraftRd'], errors='coerce')
    df['DraftRd'].fillna(100, inplace=True)  # Set NaN values to 100
    
    return df

def create_decision_trees(df):
    """
    Create decision trees for each skill group using DraftRound as the target variable.
    Only numeric features will be used.
    Accumulate accuracy scores and print them in a readable format.
    """
    trees = {}
    accuracy_scores = []  # To accumulate accuracy scores

    for skill_group, group_df in df.groupby('SkillGroup'):
        # Select only numeric columns for features (X)
        X = group_df.select_dtypes(include=[float, int])  # Selects only numeric columns
        y = group_df['DraftRd']  # Target

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Create and train the decision tree
        tree = DecisionTreeClassifier(random_state=42)
        tree.fit(X_train, y_train)

        # Make predictions and calculate accuracy
        y_pred = tree.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        # Store the tree for later use
        trees[skill_group] = tree

        # Store the accuracy score with skill group for later printing
        accuracy_scores.append((skill_group, accuracy))

    # Print accuracy scores in a readable format
    print("Accuracy Scores by Skill Group:")
    for skill_group, accuracy in accuracy_scores:
        print(f"{skill_group}: {accuracy:.4f}")  # Print with 4 decimal places for readability

    return trees


# Clean the DraftRound column
df_decision_trees = clean_draft_round(df2_sorted)

# Create decision trees for each skill group
decision_trees = create_decision_trees(df_decision_trees)

Accuracy Scores by Skill Group:
defense_heavy: 1.0000
defense_leightweight: 1.0000
defense_midweight: 1.0000
offense_heavy: 1.0000
offense_lightweight: 1.0000
offense_midweights: 1.0000
punter: 1.0000
quarterbacks: 1.0000


<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
/var/folders/fw/0c8cgnb103j_fdq5r491rhfh0000gn/T/ipykernel_24991/289216929.py:17: SyntaxWarning: invalid escape sequence '\d'
  df['DraftRd'] = df['DraftRd'].str.extract('(\d+)')
/var/folders/fw/0c8cgnb103j_fdq5r491rhfh0000gn/T/ipykernel_24991/289216929.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['DraftRd'].fillna(100, inplace=True)  # Set NaN values to 100
